# -- load data 

In [11]:
# IMPORT
# --- data sci
import pandas as pd 
import math 

# --- plots 
import matplotlib

# --- io 
import io
import requests 
import datetime
import time 

# --- local 
from importlib import reload

# -- note you'll have install this 
from messygraphs import Lending
from messygraphs.helpers import (
    get_deployments
)

In [20]:
# -- Get data from subgraphs 
# == NOTE - this is failing inside of juypter and i dont wanna figure it out - so i'm loading csvs below this to fake it. (csv files gen with same code in a .py file)

import asyncio

import pandas as pd 

# -- note you'll have install this 
from messygraphs import Lending
from messygraphs.helpers import (
    get_deployments
)

# -- define instances 
chain_data = {'balancer_v2_ethereum': {'chain': 'Ethereum', 'version': 'V2', 'protocol': 'Balancer', 'url_plug': 'balancer-v2-ethereum'}, 'bancor_v3_ethereum': {'chain': 'Ethereum', 'version': 'V3', 'protocol': 'Bancor', 'url_plug': 'bancor-v3-ethereum'}, 'platypus_finance_avalanche': {'chain': 'Avalanche', 'version': 'V1', 'protocol': 'Platypus', 'url_plug': 'platypus-finance-avalanche'}, 'curve_finance_ethereum': {'chain': 'Ethereum', 'version': 'V1', 'protocol': 'Curve', 'url_plug': 'curve-finance-ethereum'}, 'saddle_finance_ethereum': {'chain': 'Ethereum', 'version': 'V1', 'protocol': 'Saddle', 'url_plug': 'saddle-finance-ethereum'}, 'sushiswap_avalanche': {'chain': 'Avalanche', 'version': 'V1', 'protocol': 'Sushiswap', 'url_plug': 'sushiswap-avalanche'}, 'sushiswap_ethereum': {'chain': 'Ethereum', 'version': 'V1', 'protocol': 'Sushiswap', 'url_plug': 'sushiswap-ethereum'}, 'uniswap_v3_polygon': {'chain': 'Polygon', 'version': 'V3', 'protocol': 'Uniswap', 'url_plug': 'uniswap-v3-polygon'}}

In [21]:
#url = 'https://api.thegraph.com/subgraphs/name/messari/' + instance['url_plug']
chain_data_list = []

for chain, chain_details in chain_data.items(): 
    if chain in chains or len(chains) == 0: 
      # -- initialize query date
      query_dt = start_dt 

      # -- loop graph query calls until we get past the end date 
      while query_dt <= end_dt: 

        # --- get timestamp as int for the query date
        timestamp = int(datetime.datetime.timestamp( query_dt ))

        # --- define query (visit the commented urls in chain_data dictionary above to get queries) - note this is a simple query
        query_temp = f"""<financialsDailySnapshots(first: 1000, orderDirection: asc, orderBy: timestamp, where:<timestamp_gte: {timestamp}>)<
                              timestamp
                              totalValueLockedUSD
                              dailyVolumeUSD
                              dailyTotalRevenueUSD
                            >
                          >
                          """
        # --- insert brackets to finalize query format (brakets break the f string format used above)
        query_ = query_temp.replace("<", "{")
        query = query_.replace(">", "}")

        # --- call the graph endpoint
        url = 'https://api.thegraph.com/subgraphs/name/messari/' + chain['url_plug']
        graph_resp = requests.post(url, json={'query': query})
        graph_list = json.loads(graph_resp.text)['data']['financialsDailySnapshots'] # note - u may want to do some error handling before this

        # --- unpack data response and format into our pandas dataframe layout
        for item in graph_list: 
          # ---- convert timestamp to datetime object - easier to format 
          temp_dt = datetime.datetime.fromtimestamp(int(item['timestamp']))

          data_formatted = {
            'date': temp_dt.strftime("%Y-%m-%d"), 
            'timestamp': item['timestamp'], 
            'chain': chain_details['chain'],
            'protocol': 'Uniswap', 
            'protocol_version': chain_details['version'],
            'dailyVolumeUSD': round(float(item['dailyVolumeUSD']),3), 
            'dailyTVL': round(float(item['totalValueLockedUSD']),3), 
            'dailyFees': round(float(item['dailyTotalRevenueUSD']),3)
          }

          chain_data_list.append(data_formatted)
      
        # --- set last date + 1 as the new query_dt. This will pagenate and start the loop over 
        last_dt = datetime.datetime.fromtimestamp(int(graph_list[-1]['timestamp']))
        query_dt = last_dt + datetime.timedelta(days=1) # - note this should break the while loop if > end date

  # --- create dataframe for export
result_df = pd.DataFrame(chain_data_list)
result_df['date'] = pd.to_datetime(result_df['date'])
result_df.set_index('date', inplace=True)

return result_df

NameError: name 'chains' is not defined

In [13]:
financial_df = pd.concat(financial_dfs , ignore_index=True)
market_df = pd.concat(markets_dfs , ignore_index=True)

ValueError: No objects to concatenate

In [34]:
financial_df.to_csv(r'financial_df.csv', index=False)
market_df.to_csv(r'market_df.csv', index=False)

In [25]:
financial_df

,id,blockNumber,timestamp,totalValueLockedUSD,protocolControlledValueUSD,mintedTokenSupplies,dailySupplySideRevenueUSD,dailyProtocolSideRevenueUSD,dailyTotalRevenueUSD,totalDepositBalanceUSD,...,dailyDepositUSD,dailyBorrowUSD,dailyLiquidateUSD,dailyWithdrawUSD,dailyRepayUSD,date,version,chain,instance,protocol
0,18596,11363059,1606778006,0.1211387038158691701998788612961841,None,None,0.05451241671714112658994548758328286,0.006056935190793458509993943064809207,0.06056935190793458509993943064809207,0.1211387038158691701998788612961841,...,0.1211387038158691701998788612961841,0,0,0,0,2020-11-30,V2,Ethereum,aave_v2_ethereum,Aave
1,18597,11367851,1606841582,11.89956337898119954833154442456768,None,None,10.77612865202748493346936466788912,0,10.77612865202748493346936466788912,11.89956337898119954833154442456768,...,13.98942240471583788545104345752622,1.004174120453190220632081097197376,0,1.004174120453190220632081097197376,0,2020-12-01,V2,Ethereum,aave_v2_ethereum,Aave
2,18598,11376123,1606951899,11676.27657935618010547637512452716,None,None,880.2063429159296173050629493346312,110.5205408935371071911232205082187,990.7268838094667244961861698428498,11676.27657935618010547637512452716,...,12164.47320065284094001171286974372,361.7115243675556397439080951096217,0,48.68826572864025712144497975443895,65.72274404543722261312325415125642,2020-12-02,V2,Ethereum,aave_v2_ethereum,Aave
3,18599,11382777,1607039855,3713348.456118013793108654167660837,None,None,40.82765966298191935452881204993263,1.018814497008675253451569597235327,41.84647415999059460798038164716793,3713348.456118013793108654167660837,...,86841630.41529222846634195307888652,750403.1762786188118179857001126671,0,83007827.75520390651328439305115739,5172.761851802688752664776635569197,2020-12-03,V2,Ethereum,aave_v2_ethereum,Aave
4,18600,11389219,1607126126,15831169.53642713866139562788213131,None,None,170.2327583772646158572435789565403,6.562654354716201075346095664898737,176.7954127319808169325896746214391,15831169.53642713866139562788213131,...,279411101.7824755403360026639423748,915398.0916825224330769375751827159,0,264142568.0200120471122228867754785,2956.253622896821381770609494507823,2020-12-04,V2,Ethereum,aave_v2_ethereum,Aave
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,19205,18096208,1659397999,35409123.68157815008750423358400622,None,None,2045.075658707609254927715633442,578.208139585122029027655260928,2623.28379829273128395537089437,35409123.68157815008750423358400622,...,375148.93874286598496676560433805,167460.4242313907545,0,109596.57428527391532070061416311,92648.39131224215972863399865724,2022-08-01,V1,Avalanche,banker_joe_avalanche,Banker Joe
2229,19206,18138595,1659484653,35141430.41586906195041759005513788,None,None,1963.117699247453760502399079007,537.707490333711392432884733703,2500.82518958116515293528381271,35141430.41586906195041759005513788,...,782570.76682019566040386011540096,265376.78112571460891,2869.528903855860743742653152016935,849520.11626678266263161759802192,186958.86697946157322056586324807,2022-08-02,V1,Avalanche,banker_joe_avalanche,Banker Joe
2230,19207,18180138,1659568324,35352093.43747149760269029779894514,None,None,1547.091854221552541767506531065,411.035805105287175133436860795,1958.12765932683971690094339186,35352093.43747149760269029779894514,...,643695.94684155832488028884584236,181956.05848770711813324252014608,87.55291831248984543821510320448113,518515.52152381205741242722274294,194920.20899227103146460894773188,2022-08-03,V1,Avalanche,banker_joe_avalanche,Banker Joe
2231,19208,18224475,1659657519,35136788.90479844549258214229977578,None,None,2441.4072361832197339091470687365,712.3472865211678429931661185535,3153.75452270438757690231318729,35136788.90479844549258214229977578,...,885072.40810757846979574500930748,854437.55713807330713450087596143,0,671722.84171381473193691204534768,258101.01590157259504878686045797,2022-08-04,V1,Avalanche,banker_joe_avalanche,Banker Joe


# -- creating a chart: using your own df 

In [15]:
# -- create you're own df (can be any data) - usually this is some sorta groupby statement like used here 
plot_df = financial_df.groupby('date')  # -- put in millions (can make this a auto detect func)

# -- create chart object
chart = messari_chart(plot_df, layout= 'three_quarter')

# -- define chart text -- future: treat protocol name as a variable and jsut auto create a lot of this 
chart.name = 'Yule_DailyRevenueByChain_AreaPlot'
chart.title = 'Aave Daily Revenue by Broken Out by Revenue Split'
chart.subtitle = 'Previous 6 month total revenue across Aave deployments on each chain'
chart.sources = 'Messari, The Graph, Dune (@messari)'
chart.note = 'All Aave versions are included and aggregated by chain where applicable'

# -- rename columns so they print pretty 
chart.df.columns = ['Supply-Side', 'Protocol']

# -- define chart axis title 
chart.axis_label = 'Daily Revenue ($ millions)'
chart.axis_format = '${x:,.1f}M' # -- this will format in millions with one decimal place: $1.5M
chart.side_legend_decimals = 1

chart.date = chart.df.index[-1].strftime('%B %d, %Y')

# -- assign dif color set (overrides the default)
chart.color_pallet = main_colors

# -- create raw python chart 
chart.plotBasicChart(kind= 'area', legend_calc_days = 90, color_legend_chg= True)

# -- export final messari chart 
chart.createSlide()

NameError: name 'messari_chart' is not defined

# -- creating a chart: Quarterly Specific 

initial idea was we would define a block of code like this for each chart we want to create - I think some more systmizations can happen but its close 

right now, this takes the full financial_df (straight from subgraph) and plots a column in there (groupby logic and what not abstracted in the class)

In [ ]:
# =============================================================================================
# -- REVENUE BY CHAIN: plot area timeseries of total revenue by chain
# =============================================================================================

metric = 'dailyTotalRevenueUSD' # -- define the metric we are plotting. this must match a df column 
chart_type = 'area'             # -- define the chart type: 'area', 'line', 'bar'
group_column = 'chain'          # -- define which column we are grouping. If doing an area chart by chain, this would be the chain column name 
unit_adj = 1000000              # -- set to millions 
moving_avg = 7                  # -- 7d moving avg to smooth the plot 
layout = 'three_quarter'        # -- three_quarter includes a side bar legend w/ values. other options incld ['full_width', 'full_width_bottom', 'three_quarter']

# -- define messari chart and text components 
revenue_area_side = messari_chart(financial_df, layout= layout)

revenue_area_side.name = 'DailyRevenueByChain_AreaPlot'
revenue_area_side.title = 'Aave Daily Revenue by Chain'
revenue_area_side.subtitle = 'Previous 6 month total revenue across Aave deployments on each chain'
revenue_area_side.sources = 'Messari, The Graph, Dune (@messari)'
revenue_area_side.note = 'All Aave versions are included and aggregated by chain where applicable'

revenue_area_side.axis_label = 'Daily Revenue ($ millions)'
revenue_area_side.axis_format = '${x:,.1f}M' # -- this will format in millions with one decimal place: $1.5M
revenue_area_side.side_legend_decimals = 2

revenue_area_side.color_pallet = main_colors # -- test out new color pallet bc i hate the current one (this overrides)

# -- auto filter dataframe to last quarter with a minimum of 6 quarters ago (basically wraps some groupby logic)
revenue_area_side.filterDataByCatagory( metric= metric, 
                                        category_column = group_column, # -- group by chain. can group by other columns like version, token, etc provided its a df column 
                                        unit_adjustment = unit_adj,     # -- adjust the untis of the metric 
                                        moving_avg = moving_avg,        # -- set a moving avgerage (in days)
                                        order_method = 'sum',           # -- 'sum' will order columns + legend by the sum of the metric through the quarter. 'last' will use quarter-end values to sort 
                                        auto_select_quarter = True)     # -- True will assume data pulled after the quarter and find the previous quarter. If False, need to specify min and max timestamps at object level (or in df filter level)

# -- set the date to use in the chart now that the df is filtered (logic block above filters the raw )
revenue_area_side.date = revenue_area_side.df.index[-1].strftime('%B %d, %Y')

# -- create raw python chart 
revenue_area_side.plotBasicChart(kind= chart_type, legend_calc_days = 90, color_legend_chg= True)

# -- export final messari chart 
revenue_area_side.createSlide()

# -- Creating a Table: Quarterly Specific 

this is the most complex part - building tables. This works by defining a base chart template background and creating rows 

each row can use the main df of the object or have a custom df (good for data not in a subgraph) - there's a ton of other settings like spacing, font size, font, etc. see below

In [11]:
# -- create a KPI Table 
layout = 'quarterly_table' # -- this controls the initialization of the chart object to load table settings vs chart settings 

table = messari_chart(financial_df, layout=layout)

table.name = 'AaveQuarterly_Kpi_new'
table.title = 'Aave Quarterly KPIs'
table.subtitle = 'Key Indicators for Aave business operations'
table.sources = 'Messari, The Graph, Aave, Dune (@messari)'
table.note = 'All Aave versions are included and aggregated by chain where applicable'

table.table_config['row_spacing'] = 52

table.rows = {
  'outstanding_loans':   {'display_name': 'Outstanding Loans','df_column': 'totalBorrowBalanceUSD', 'value_format': "${:,.1f}", 'calc_method': 'last'}, 
  'outstanding_depotis': {'display_name': 'Outstanding Deposits','df_column': 'totalDepositBalanceUSD', 'value_format': "${:,.1f}", 'calc_method': 'last'}, 
  'originations': {'display_name': 'Quarterly Originations','df_column': 'dailyBorrowUSD', 'value_format': "${:,.1f}", 'calc_method': 'sum'}, 
  'new_deposits': {'display_name': 'Quarterly Deposits','df_column': 'dailyDepositUSD', 'value_format': "${:,.1f}", 'calc_method': 'sum'}, 
  'unquie_addrs': {'display_name': 'Unique Addresses', 'df': unique_df, 'df_column': 'user_address', 'value_format': "{:,.0f}", 'calc_method': 'sum', 'denomination': 1}, 
  'roa': {'display_name': 'Return on Assets','df_column': 'roa', 'value_format': "{0:.1%}", 'calc_method': 'avg', 'denomination': 1} 
}

table.buildTable(auto_select_quarter = True) # -- auto select quarter basically assumes 1) you want quarterly periods and 2) the df was collected after the quarter you are interested in (i.e. you dont need to filter)
table.createSlide()

NameError: name 'messari_chart' is not defined

In [ ]:
# -- create income statement (show off some of the formatting stuff not included in the above)
# == NOTE this will break - if you want to see it work, ask dustin for the input df code for grants and inititives 

table = messari_chart(financial_df, layout=layout)

table.name = 'AaveQuarterly_IncomeStatement'
table.title = 'Aave Quarterly Income Statement'
table.subtitle = 'Quarter over quarter financial statements'
table.sources = 'Messari, The Graph, Aave, Dune (@messari)'
table.note = 'All Aave versions are included and aggregated by chain where applicable'

table.table_config['row_spacing'] = 36

table.rows = {
  'interest_revenue': {'display_name': 'Interest Revenue',     'df_column': 'dailyTotalRevenueUSD', 'value_format': "${:,.1f}", 'calc_method': 'sum', 'heading_font': 'Montserrat-Reg', 'contribute_topline': True}, 
  #'liquidation_revenue': {'display_name': 'Outstanding Deposits','df_column': None, 'value_format': "${:,.1f}", 'calc_method': 'last', 'heading_font': 'Montserrat-Reg'}, 
  'depositor_claims': {'display_name': '(-) Depositor Claims',  'df_column': 'dailySupplySideRevenueUSD', 'value_format': "${:,.1f}", 'calc_method': 'sum', 'heading_font': 'Montserrat-Reg', 'sign': -1}, 
  'gross_revenue':    {'display_name': 'Gross Protocol Revenue','df_column': 'dailyProtocolSideRevenueUSD', 'value_format': "${:,.1f}", 'calc_method': 'subtotal_rows', 'subtotal_rows': [0,1]}, 
  'gross_margin':     {'display_name': 'Margin',                'df_column': 'none', 'value_format': "{0:.1%}", 'calc_method': 'margin', 'denomination': 1, 'heading_font': 'Open-Sans-Italic'}, 
  #'treasury_expenses': {'display_name': '(-) Treasury Expenses','df_column': 'roa', 'value_format': "{0:.1%}", 'calc_method': 'avg', 'denomination': 1},
  'grants_issued':    {'display_name': '(-) Grants Issued','df': grants_df,'df_column': 'Amount Paid', 'value_format': "${:,.1f}", 'calc_method': 'sum', 'sign': -1, 'heading_font': 'Montserrat-Reg'}, 
  'net_income':       {'display_name': 'Net Income','df_column': 'none', 'value_format': "${:,.1f}", 'calc_method': 'subtotal_rows', 'subtotal_rows': [2,4]}, 
  'net_margin':       {'display_name': 'Margin', 'df_column': 'none', 'value_format': "{0:.1%}", 'calc_method': 'margin', 'denomination': 1, 'heading_font': 'Open-Sans-Italic'}, 
  'token_incentives': {'display_name': 'Token Incentives','df': supply_df_q.fillna(0), 'df_column': 'supply_chg', 'value_format': "${:,.1f}", 'calc_method': 'last', 'heading_font': 'Montserrat-Reg', 'sign': -1}, 
  'per_circ_supply':  {'display_name': 'Percent of Circulating Supply', 'df': supply_df_q.fillna(0),  'df_column': 'per_supply', 'value_format': "{0:.1%}", 'calc_method': 'last', 'denomination': 1, 'heading_font': 'Open-Sans-Italic', 'row_spacing': int(table.table_config['row_spacing']*0.25), 'font_size': int(table.table_config['font_size'] * 0.75)}, 
  'net_adj_revenue':  {'display_name': 'Adjusted Net Income','df_column': 'dailyDepositUSD', 'value_format': "${:,.1f}",'calc_method': 'subtotal_rows', 'subtotal_rows': [5,7]}, 
  'net_adj_margin':   {'display_name': 'Margin', 'df_column': 'user_address', 'value_format': "{0:.1%}", 'calc_method': 'margin', 'denomination': 1, 'heading_font': 'Open-Sans-Italic'}
}

table.buildTable(auto_select_quarter = True) # -- auto select quarter basically assumes 1) you want quarterly periods and 2) the df was collected after the quarter you are interested in (i.e. you dont need to filter)
table.createSlide()

In [ ]:
rows = {
  'row name': {
    'display_name': '',        # -- str: [REQUIRED]the row header printed on the table 
    'font': '',                # -- str: font name as a string. 1) must match the dict keys of table object font var 2) only used to overide table config for specific row 
    'font_size': '',           # -- int: font size. used only to overide the table default for a specific row 
    'row_spacing': '',         # -- int: number of pixels of spacing between previous row 
    'sign': '',                # -- int: controls whether a value is added or subtracted from a subtotal. either -1 or 1. defaults to 1 so dont need to pass
    'denomination': '',        # -- str: controls what to divide the value by. either thousands, millions, or billions. uses table default if none provided. passing anything else will force it to use 1 (i.e not redenominate)
    'value_format': '',        # -- str: specific format for the value. example: "${x:,.1f}" will format in dollars with a single decimal
    'temp_df': '',             # -- df:  pandas df to calc the value from. table default used if not provided 
    'df_column': '',           # -- str: [REQUIRED] the column header in the df of the value calculated in the row 
    'calc_method': '',         # -- str: controls how to calculate the value for the period. options = ['sum', 'avg', 'last', 'margin']. Sum will sum value for the quarter, last gets value on last day of quarter, and margin says calc from table values instead
    'contribute_topline': '',  # -- bool: controls whether the value is counted in the top line of the column. used for margin calculations where the top line total is the denominator
    'y': '',                   # -- int: manual y coordinate placement for the row (use if you dont wanna figure out a row location from spacing)
    'placeholder__': '', 
  }
}

columns = {
  'col_name': {
    'padding': 8,          # -- int: amount of pixel padding to be added between the column right edge and the value 
    'col_width': '',       # -- int: overrides the default column width in pixels 
    'placeholder__': ''
  }
}

# -- delete later 

In [ ]:
sol_df = pd.read_csv('solana dex users.csv')

In [ ]:
sol_df.columns

In [ ]:
plot_df = sol_df.groupby(['day','DEX'])['dau'].sum().unstack()
plot_df.sort_index() 

In [ ]:
new_cols = list(plot_df.sum().sort_values(ascending = False).index)
plot_df = plot_df[new_cols[:-4]]

plot_df

In [ ]:

sol_df.set_index('day')

In [ ]:
main_colors = ['#1B80F3','#00D1FF','#1BF38C','#9766FF','#F3D64E','#E76BA6','#F2A258','#7CB5F6','#81E8FF','#89EABB','#B698F8','#FEAFD4','#F5E391','#F3B09B']


In [ ]:
# -- create you're own df (can be any data) - usually this is some sorta groupby statement like used here 
#plot_df = financial_df.groupby('date')[['dailySupplySideRevenueUSD', 'dailyProtocolSideRevenueUSD']].sum().rolling(7).mean() / 1000000 # -- put in millions (can make this a auto detect func)

# -- create chart object
chart = messari_chart(plot_df, layout= 'full_width_bottom')

# -- define chart text -- future: treat protocol name as a variable and jsut auto create a lot of this 
chart.name = 'solana_dex_users'
chart.title = "StepN's Daily Active Users Dominate Solana DEXs"
chart.subtitle = 'Daily active users across Solana DEXs'
chart.sources = 'Dune (@twocrows)'
chart.note = 'StepN switched from using Orca as their DEX to using their own'


# -- define chart axis title 
chart.axis_label = 'Daily Active DEX Users'
chart.axis_format = '{x:,.0f}' # -- this will format in millions with one decimal place: $1.5M
chart.side_legend_decimals = 1

chart.date = 'July 29, 2022'

# -- assign dif color set (overrides the default)
chart.color_pallet = main_colors

chart.loc_y = 75
# -- create raw python chart 
chart.plotBasicChart(kind= 'bar', color_legend_chg= True, stacked=True, group_method='daily_bar')

# -- export final messari chart 
chart.createSlide()

In [ ]:
chart.loc_y

In [ ]:
ax = plot_df.plot.bar(stacked=True, figsize=(10,5))

# Create list of monthly timestamps by selecting the first weekly timestamp of each
# month (in this example, the first Sunday of each month)
monthly_timestamps = [timestamp for idx, timestamp in enumerate(plot_df.index)
                      if (timestamp.month != plot_df.index[idx-1].month) | (idx == 0)]

# Automatically select appropriate number of timestamps so that x-axis does
# not get overcrowded with tick labels
step = 1
while len(monthly_timestamps[::step]) > 10: # increase number if time range >3 years
    step += 1
timestamps = monthly_timestamps[::step]

# Create tick labels from timestamps
labels = [ts.strftime('%b\n%Y') if ts.year != timestamps[idx-1].year
          else ts.strftime('%b') for idx, ts in enumerate(timestamps)]

# Set major ticks and labels
ax.set_xticks([plot_df.index.get_loc(ts) for ts in timestamps])
ax.set_xticklabels(labels)

# Set minor ticks without labels
ax.set_xticks([plot_df.index.get_loc(ts) for ts in monthly_timestamps], minor=True)

# Rotate and center labels
ax.figure.autofmt_xdate(rotation=0, ha='center')

In [ ]:
import matplotlib.dates as mdates
import numpy as np

In [ ]:
# Create pandas stacked bar chart with the default bar width = 0.5
ax = plot_df.plot.bar(stacked=True, figsize=(10,5))

# Compute width of bars in matplotlib date units, 'md' (in days) and adjust it if
# the bar width in df.plot.bar has been set to something else than the default 0.5
bar_width_md_default, = np.diff(mdates.date2num(plot_df.index[:2]))/2
bar_width = ax.patches[0].get_width()
bar_width_md = bar_width*bar_width_md_default/0.5

# Compute new x values in matplotlib date units for the patches (rectangles) that
# make up the stacked bars, adjusting the positions according to the bar width:
# if the frequency is in months (or years), the bars may not always be perfectly
# centered over the tick marks depending on the number of days difference between
# the months (or years) given by df.index[0] and [1] used to compute the bar 
# width, this should not be noticeable if the bars are wide enough.
x_bars_md = mdates.date2num(plot_df.index) - bar_width_md/2
nvar = len(ax.get_legend_handles_labels()[1])
x_patches_md = np.ravel(nvar*[x_bars_md])

# Set bars to new x positions and adjust width: this loop works fine with NaN
# values as well because in bar plot NaNs are drawn with a rectangle of 0 height
# located at the foot of the bar, you can verify this with patch.get_bbox()
for patch, x_md in zip(ax.patches, x_patches_md):
    patch.set_x(x_md)
    patch.set_width(bar_width_md)

# Set major ticks
maj_loc = mdates.AutoDateLocator()
ax.xaxis.set_major_locator(maj_loc)

# Show minor tick under each bar (instead of each month) to highlight
# discrepancy between major tick locator and bar positions seeing as no tick
# locator is available for first-week-of-the-month frequency
ax.set_xticks(x_bars_md + bar_width_md/2, minor=True)

# Set major tick formatter
zfmts = ['', '%b\n%Y', '%b', '%b-%d', '%H:%M', '%H:%M']
fmt = mdates.ConciseDateFormatter(maj_loc, zero_formats=zfmts, show_offset=False)
ax.xaxis.set_major_formatter(fmt)

# Shift the plot frame to where the bars are now located
xmin = min(x_bars_md) - bar_width_md
xmax = max(x_bars_md) + 2*bar_width_md
ax.set_xlim(xmin, xmax)

# Adjust tick label format last, else it may sometimes not be applied correctly
ax.figure.autofmt_xdate(rotation=0, ha='center')